# Simple CNN Model

## Import packages

In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import shutil
from PIL import Image
import tensorflow
from tensorflow.keras import layers,models
from tensorflow.keras.callbacks import EarlyStopping

## Import data

In [32]:
data = pd.read_csv("../Data_analysis/df_binary.csv")

In [3]:
data.head()

,Unnamed: 0,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,...,C,A,H,M,O,filepath,labels,target,filename,target_binary
0,0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,...,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg,0
1,1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,...,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg,0
2,2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,...,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg,1
3,3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,...,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg,1
4,4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,...,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg,1


In [34]:
source = "../raw_data/preprocessed_images"
dest = "../data" 

for _, row in data.iterrows():
    source_file = os.path.join(source,row['filename'])
    dest_file = os.path.join(dest,str(row['target_binary']), row['filename'])
    shutil.copy(source_file, dest_file)

In [4]:
#data_path = "../raw_data/preprocessed_images/"
#imgs = []
images_path = [elt for elt in os.listdir(data_path) if elt.find('.jpg')>0]
for file in data.filename:
    path = os.path.join(data_path, file)
    if os.path.exists(path):
        image = Image.open(path)
        imgs.append(np.array(image))

In [5]:
X = np.array(imgs)
y = data['target_binary']

In [6]:
X.shape, y.shape

((6392, 512, 512, 3), (6392,))

In [9]:
split = int(len(imgs) /6.)
X_test, X_train = X[:split], X[split:]
y_test, y_train = y[:split], y[split:]

In [13]:
X_test.shape, y_test.shape, X_train.shape, y_train.shape

((1065, 512, 512, 3), (1065,), (5327, 512, 512, 3), (5327,))

In [18]:
y_test = np.array(y_test).reshape(-1,1)
y_train = np.array(y_train).reshape(-1,1)

In [19]:
y_test.shape, y_train.shape

((1065, 1), (5327, 1))

In [28]:
np.unique(y_train)

array([0, 1])

## CNN Model

In [29]:
def load_own_model():
    model = models.Sequential()
    model.add(layers.Rescaling(1/255, input_shape = (512,512,3)))
    model.add(layers.Conv2D(16, (5,5), activation="relu", padding="same", strides=(1,1) ))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    
    model.add(layers.Conv2D(32, (4,4), activation="relu", padding="same", strides=(1,1) ))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    
    model.add(layers.Conv2D(64, (3,3), activation="relu", padding="same", strides=(1,1) ))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(126, activation="relu"))
    model.add(layers.Dense(1, activation="sigmoid"))
    
    model.compile(optimizer="adam",
                loss="binary_crossentropy",
                metrics=["accuracy"])
    return model

In [30]:
model = load_own_model()
model.fit(X_train, y_train, epochs=100, batch_size=32, 
          validation_split=0.3, callbacks=[EarlyStopping(patience=20, restore_best_weights=True)])

Epoch 1/100
117/117 [==============================] - 326s 3s/step - loss: 0.7607 - accuracy: 0.5864 - val_loss: 1.1630 - val_accuracy: 0.3346
Epoch 2/100
117/117 [==============================] - 393s 3s/step - loss: 0.6627 - accuracy: 0.6113 - val_loss: 0.8124 - val_accuracy: 0.3377
Epoch 3/100
117/117 [==============================] - 335s 3s/step - loss: 0.6535 - accuracy: 0.6360 - val_loss: 0.7348 - val_accuracy: 0.3652
Epoch 4/100
117/117 [==============================] - 340s 3s/step - loss: 0.6465 - accuracy: 0.6371 - val_loss: 0.8354 - val_accuracy: 0.3690
Epoch 5/100
117/117 [==============================] - 331s 3s/step - loss: 0.6399 - accuracy: 0.6443 - val_loss: 0.8175 - val_accuracy: 0.4028
Epoch 6/100
117/117 [==============================] - 331s 3s/step - loss: 0.6353 - accuracy: 0.6475 - val_loss: 0.7891 - val_accuracy: 0.3740
Epoch 7/100
117/117 [==============================] - 331s 3s/step - loss: 0.6308 - accuracy: 0.6508 - val_loss: 0.8382 - val_accuracy:

KeyboardInterrupt: 